In [1]:
!pip install gensim==3.8.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scattertext 0.1.7 requires gensim>=4.0.0, but you have gensim 3.8.3 which is incompatible.


In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from wordcloud import WordCloud, ImageColorGenerator
from gensim.summarization.textcleaner import split_sentences
import nltk
import string
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load data

In [3]:
train_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_train_en.xlsx")
test_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_test_en.xlsx")

In [4]:
print(train_set.shape)
print(test_set.shape)

(304768, 2)
(11529, 2)


In [5]:
train_set.describe()

,Document,Summary
count,304768,304768
unique,304739,304112
top,Please also ensure you follow current coronavi...,With the world's biggest bike race starting in...
freq,19,30


In [6]:
test_set.describe()

,Document,Summary
count,11529,11529
unique,11529,11521
top,"But Eluned Morgan conceded that it would be ""d...",Here are five things you need to know about th...
freq,1,2


# EDA

## some statistics 

In [7]:
# the number of sentences in the Summary
## TRAIN
train_set['sum_sent_count'] = train_set['Summary'].apply(lambda x : len(split_sentences(x)))
#  the number of sentences in the text
train_set['text_sent_count'] = train_set['Document'].apply(lambda x : len(split_sentences(x)))

## TEST
# the number of sentences in the Summary
test_set['sum_sent_count'] = test_set['Summary'].apply(lambda x : len(split_sentences(x)))
#  the number of sentences in the text
test_set['text_sent_count'] = test_set['Document'].apply(lambda x : len(split_sentences(x)))


In [8]:
## TRAIN
# the number of word in the summary
train_set['sum_word_count'] = train_set['Summary'].apply(lambda x : len(x.split()))
# the number of word in the text
train_set['text_word_count'] = train_set['Document'].apply(lambda x : len(x.split()))

## TEST
test_set['sum_word_count'] = test_set['Summary'].apply(lambda x : len(x.split()))
# the number of word in the text
test_set['text_word_count'] = test_set['Document'].apply(lambda x : len(x.split()))

In [9]:
## TRAIN
# the number of characters in the summary
train_set['sum_char_count'] = train_set['Summary'].apply(lambda x : len(x.replace(" ","")))
#the number of characters in the text
train_set['text_char_count'] = train_set['Document'].apply(lambda x : len(x.replace(" ","")))

## TEST
# the number of characters in the summary
test_set['sum_char_count'] = test_set['Summary'].apply(lambda x : len(x.replace(" ","")))
#the number of characters in the text
test_set['text_char_count'] = test_set['Document'].apply(lambda x : len(x.replace(" ","")))

In [10]:
## TRAIN
# the density of words in the summary
train_set['sum_word_density'] = train_set['sum_word_count'] / (train_set['sum_char_count'] + 1)
#  the density of words in the text
train_set['text_word_density'] = train_set['text_word_count'] / (train_set['text_char_count'] + 1)

## TEST
# the density of words in the summary
test_set['sum_word_density'] = test_set['sum_word_count'] / (test_set['sum_char_count'] + 1)
#  the density of words in the text
test_set['text_word_density'] = test_set['text_word_count'] / (test_set['text_char_count'] + 1)


In [11]:
## TRAIN
# the density of sentences in the summary
train_set['sum_sent_density'] = train_set['sum_sent_count'] / (train_set['sum_word_count'] + 1)
# the density of sentences in the text
train_set['text_sent_density'] = train_set['text_sent_count'] / (train_set['text_word_count'] + 1)


## TEST
# the density of sentences in the summary
test_set['sum_sent_density'] = test_set['sum_sent_count'] / (test_set['sum_word_count'] + 1)
# the density of sentences in the text
test_set['text_sent_density'] = test_set['text_sent_count'] / (test_set['text_word_count'] + 1)


In [12]:
def count_stopwords(text):
    ''' Return the number of stopwords in the text
        Input:
            - text: string
            - stopwords: list of string, containing the stopwords
        Output:
            - int, number of stopwords in the text argument
    '''
    word_tokens =nltk.word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in  set(stopwords.words('english'))]
    
    return len(stopwords_x)

In [13]:
## TRAIN
# the number of punctuations in the summary
train_set['sum_punc_count'] = train_set['Summary'].apply(lambda x : len([a for a in x if a in string.punctuation]))
# the number of punctuations in the text
train_set['text_punc_count'] = train_set['Document'].apply(lambda x : len([a for a in x if a in string.punctuation]))

## TEST
# the number of punctuations in the summary
test_set['sum_punc_count'] = test_set['Summary'].apply(lambda x : len([a for a in x if a in string.punctuation]))
# the number of punctuations in the text
test_set['text_punc_count'] = test_set['Document'].apply(lambda x : len([a for a in x if a in string.punctuation]))


In [14]:
train_set.describe()

,sum_sent_count,text_sent_count,sum_word_count,text_word_count,sum_char_count,text_char_count,sum_word_density,text_word_density,sum_sent_density,text_sent_density,sum_punc_count,text_punc_count
count,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000,304768.000000
mean,1.078939,20.416241,22.219797,459.682267,110.158983,2263.673624,0.200742,0.202349,0.048564,0.045193,2.773890,76.721303
std,0.347851,16.006447,7.230242,335.410688,35.059637,1621.047225,0.022767,0.010405,0.013695,0.008542,1.780912,60.081099
min,1.000000,1.000000,5.000000,32.000000,40.000000,169.000000,0.086207,0.118519,0.010870,0.000409,0.000000,0.000000
25%,1.000000,11.000000,18.000000,241.000000,89.000000,1192.000000,0.185185,0.195407,0.040000,0.039370,1.000000,38.000000
50%,1.000000,16.000000,21.000000,366.000000,107.000000,1814.000000,0.200000,0.201916,0.045455,0.044386,2.000000,61.000000
75%,1.000000,26.000000,25.000000,604.000000,122.000000,2981.000000,0.215190,0.208827,0.055556,0.050000,4.000000,100.000000
max,11.000000,529.000000,172.000000,6366.000000,797.000000,28130.000000,0.388889,0.271523,0.846154,0.335873,28.000000,1604.000000


In [15]:
test_set.describe()

,sum_sent_count,text_sent_count,sum_word_count,text_word_count,sum_char_count,text_char_count,sum_word_density,text_word_density,sum_sent_density,text_sent_density,sum_punc_count,text_punc_count
count,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000
mean,1.035736,19.218666,21.241651,437.198630,105.640298,2158.596669,0.200129,0.202112,0.048507,0.044634,2.629109,72.849163
std,0.203474,11.957447,4.753051,262.373004,22.933391,1279.351023,0.022786,0.010113,0.012770,0.007927,1.555699,46.497760
min,1.000000,2.000000,8.000000,61.000000,43.000000,290.000000,0.125000,0.145057,0.023810,0.003135,0.000000,4.000000
25%,1.000000,11.000000,18.000000,246.000000,89.000000,1221.000000,0.184211,0.195243,0.040000,0.039301,1.000000,40.000000
50%,1.000000,16.000000,21.000000,360.000000,106.000000,1783.000000,0.200000,0.201726,0.045455,0.043977,2.000000,60.000000
75%,1.000000,24.000000,24.000000,574.000000,121.000000,2828.000000,0.214815,0.208381,0.055556,0.049296,3.000000,94.000000
max,5.000000,113.000000,42.000000,1667.000000,220.000000,8344.000000,0.310811,0.244517,0.277778,0.157233,15.000000,444.000000
